# Improvements to a Neural Network

In a previous notebook we implemented a simple feedforward neural network and used it to classify handwritten digits. In this notebook we modify the simple network, adding several improvements which improve the accuracy of the network. As before, the code in this notebook is an adaption of the code (network2.py) included with the book "[Neural Networks and Deep Learning](http://neuralnetworksanddeeplearning.com/)" by Michael Nielsen, specifically the [version and adapted to work with Python 3](https://github.com/MichalDanielDobrzanski/DeepLearningPython) by Michal Daniel Dobrzanski.

## Implementation of the Network

### Helpful libraries and convenience functions

First, we load some standard Python libraries.

In [1]:
import json
import random
import sys
import numpy as np

Define a convenience function for identifying which digit the network recognised.

In [2]:
def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the j'th position
    and zeroes elsewhere.  This is used to convert a digit (0...9)
    into a corresponding desired output from the neural network.

    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

A function for loading a previously-saved network.

In [3]:
def load(filename):
    """Load a neural network from the file ``filename``.  Returns an
    instance of Network.

    """
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net

### Activation function

We now define our activation function and another function to compute its derivative

In [4]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

### Cost functions

We will be adding support in our Network class for different choices of cost function. For this, a cost function should be defined as a class  with two functions:
* fn(a, y): compute the cost for a given a and y
* delta: compute the error, $\delta^L$ in the output layer.

We now define two specific cost functions

#### Quadratic cost

In [5]:
class QuadraticCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.

        """
        return 0.5*np.linalg.norm(a-y)**2

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer."""
        return (a-y) * sigmoid_prime(z)

####  Cross-entropy cost

In [6]:
class CrossEntropyCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.  Note that np.nan_to_num is used to ensure numerical
        stability.  In particular, if both ``a`` and ``y`` have a 1.0
        in the same slot, then the expression (1-y)*np.log(1-a)
        returns nan.  The np.nan_to_num ensures that that is converted
        to the correct value (0.0).

        """
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer.  Note that the
        parameter ``z`` is not used by the method.  It is included in
        the method's parameters in order to make the interface
        consistent with the delta method for other cost classes.

        """
        return (a-y)

### Network class

Next, we define a class "Network" that will represent our neural network. This includes several modifications to the previous class, primarily to allow for different cost functions, regularization, better weight initialization, and early stopping.

In [7]:
class Network(object):

    def __init__(self, sizes, cost=CrossEntropyCost):
        """The list ``sizes`` contains the number of neurons in the respective
        layers of the network.  For example, if the list was [2, 3, 1]
        then it would be a three-layer network, with the first layer
        containing 2 neurons, the second layer 3 neurons, and the
        third layer 1 neuron.  The biases and weights for the network
        are initialized randomly, using
        ``self.default_weight_initializer`` (see docstring for that
        method).

        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost=cost

    def default_weight_initializer(self):
        """Initialize each weight using a Gaussian distribution with mean 0
        and standard deviation 1 over the square root of the number of
        weights connecting to the same neuron.  Initialize the biases
        using a Gaussian distribution with mean 0 and standard
        deviation 1.

        Note that the first layer is assumed to be an input layer, and
        by convention we won't set any biases for those neurons, since
        biases are only ever used in computing the outputs from later
        layers.

        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initializer(self):
        """Initialize the weights using a Gaussian distribution with mean 0
        and standard deviation 1.  Initialize the biases using a
        Gaussian distribution with mean 0 and standard deviation 1.

        Note that the first layer is assumed to be an input layer, and
        by convention we won't set any biases for those neurons, since
        biases are only ever used in computing the outputs from later
        layers.

        This weight and bias initializer uses the same approach as in
        Chapter 1, and is included for purposes of comparison.  It
        will usually be better to use the default weight initializer
        instead.

        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0,
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False,
            early_stopping_n = 0):
        """Train the neural network using mini-batch stochastic gradient
        descent.  The ``training_data`` is a list of tuples ``(x, y)``
        representing the training inputs and the desired outputs.  The
        other non-optional parameters are self-explanatory, as is the
        regularization parameter ``lmbda``.  The method also accepts
        ``evaluation_data``, usually either the validation or test
        data.  We can monitor the cost and accuracy on either the
        evaluation data or the training data, by setting the
        appropriate flags.  The method returns a tuple containing four
        lists: the (per-epoch) costs on the evaluation data, the
        accuracies on the evaluation data, the costs on the training
        data, and the accuracies on the training data.  All values are
        evaluated at the end of each training epoch.  So, for example,
        if we train for 30 epochs, then the first element of the tuple
        will be a 30-element list containing the cost on the
        evaluation data at the end of each epoch. Note that the lists
        are empty if the corresponding flag is not set.

        """

        # early stopping functionality:
        best_accuracy=1

        training_data = list(training_data)
        n = len(training_data)

        if evaluation_data:
            evaluation_data = list(evaluation_data)
            n_data = len(evaluation_data)

        # early stopping functionality:
        best_accuracy=0
        no_accuracy_change=0

        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(
                    mini_batch, eta, lmbda, len(training_data))

            print("Epoch %s training complete" % j)

            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("Cost on training data: {}".format(cost))
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print("Accuracy on training data: {} / {}".format(accuracy, n))
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print("Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("Accuracy on evaluation data: {} / {}".format(self.accuracy(evaluation_data), n_data))

            # Early stopping:
            if early_stopping_n > 0:
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    no_accuracy_change = 0
                    #print("Early-stopping: Best so far {}".format(best_accuracy))
                else:
                    no_accuracy_change += 1

                if (no_accuracy_change == early_stopping_n):
                    #print("Early-stopping: No accuracy change in last epochs: {}".format(early_stopping_n))
                    return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy

        return evaluation_cost, evaluation_accuracy, \
            training_cost, training_accuracy

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        """Update the network's weights and biases by applying gradient
        descent using backpropagation to a single mini batch.  The
        ``mini_batch`` is a list of tuples ``(x, y)``, ``eta`` is the
        learning rate, ``lmbda`` is the regularization parameter, and
        ``n`` is the total size of the training data set.

        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def accuracy(self, data, convert=False):
        """Return the number of inputs in ``data`` for which the neural
        network outputs the correct result. The neural network's
        output is assumed to be the index of whichever neuron in the
        final layer has the highest activation.

        The flag ``convert`` should be set to False if the data set is
        validation or test data (the usual case), and to True if the
        data set is the training data. The need for this flag arises
        due to differences in the way the results ``y`` are
        represented in the different data sets.  In particular, it
        flags whether we need to convert between the different
        representations.  It may seem strange to use different
        representations for the different data sets.  Why not use the
        same representation for all three data sets?  It's done for
        efficiency reasons -- the program usually evaluates the cost
        on the training data and the accuracy on other data sets.
        These are different types of computations, and using different
        representations speeds things up.  More details on the
        representations can be found in
        mnist_loader.load_data_wrapper.

        """
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                       for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in data]

        result_accuracy = sum(int(x == y) for (x, y) in results)
        return result_accuracy

    def total_cost(self, data, lmbda, convert=False):
        """Return the total cost for the data set ``data``.  The flag
        ``convert`` should be set to False if the data set is the
        training data (the usual case), and to True if the data set is
        the validation or test data.  See comments on the similar (but
        reversed) convention for the ``accuracy`` method, above.
        """
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
        cost += 0.5*(lmbda/len(data))*sum(np.linalg.norm(w)**2 for w in self.weights) # '**' - to the power of.
        return cost

    def save(self, filename):
        """Save the neural network to the file ``filename``."""
        data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.close()

## Example

We now look at a simple example, running the new network with the quadratic cost function, no regularization and the original method for initializing the weights. First, we load the training data:

In [8]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [9]:
net = Network([784, 30, 10], cost=QuadraticCost)
net.large_weight_initializer()
net.SGD(training_data, 10, 10, 3.0, evaluation_data=test_data, monitor_evaluation_accuracy=True)

Epoch 0 training complete
Accuracy on evaluation data: 8264 / 10000
Epoch 1 training complete
Accuracy on evaluation data: 8370 / 10000
Epoch 2 training complete
Accuracy on evaluation data: 8476 / 10000
Epoch 3 training complete
Accuracy on evaluation data: 9300 / 10000
Epoch 4 training complete
Accuracy on evaluation data: 9306 / 10000
Epoch 5 training complete
Accuracy on evaluation data: 9378 / 10000
Epoch 6 training complete
Accuracy on evaluation data: 9368 / 10000
Epoch 7 training complete
Accuracy on evaluation data: 9382 / 10000
Epoch 8 training complete
Accuracy on evaluation data: 9397 / 10000
Epoch 9 training complete
Accuracy on evaluation data: 9414 / 10000


([], [8264, 8370, 8476, 9300, 9306, 9378, 9368, 9382, 9397, 9414], [], [])

## Exercises

### Cross-entropy cost function
1. Train network with a single hidden layer with 30 neurons using the cross-entropy cost function. Note: you should use a different learning rate to produce comparable results. A good rule of thumb is to reduce the rate by a factor of 6 when using a cross-entropy cost function compared to a quadratic cost function.
2. Compare the accuracy against the quadratic cost case.
3. Repeat, but with 100 neurons.

### Overfitting and regularization
1. Train a 30 neuron network with a 1000-sample subset of the MNIST training data. Use the cross-entropy cost function and train for 400 epochs. Enabe monitoring of the evaluation accuracy and training cost and plot the results.
2. Train the network again, but this time including regularization by setting `lmbda=0.1`.
3. Compare the results in the two cases.


### Weight initialization
1. Train your network again, but this time use the improved `default_weight_initializer` weight initialization.
2. Compare the accuracy with previous cases.

In [10]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [11]:
net = Network([784, 30, 10], cost=CrossEntropyCost)
net.large_weight_initializer()
net.SGD(training_data, 10, 10, 0.5, evaluation_data=test_data, monitor_evaluation_accuracy=True)

Epoch 0 training complete
Accuracy on evaluation data: 9129 / 10000
Epoch 1 training complete
Accuracy on evaluation data: 9271 / 10000
Epoch 2 training complete
Accuracy on evaluation data: 9374 / 10000
Epoch 3 training complete
Accuracy on evaluation data: 9397 / 10000
Epoch 4 training complete
Accuracy on evaluation data: 9438 / 10000
Epoch 5 training complete
Accuracy on evaluation data: 9440 / 10000
Epoch 6 training complete
Accuracy on evaluation data: 9424 / 10000
Epoch 7 training complete
Accuracy on evaluation data: 9455 / 10000
Epoch 8 training complete
Accuracy on evaluation data: 9482 / 10000
Epoch 9 training complete
Accuracy on evaluation data: 9462 / 10000


([], [9129, 9271, 9374, 9397, 9438, 9440, 9424, 9455, 9482, 9462], [], [])

In [14]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net = Network([784, 100, 10], cost=QuadraticCost)
net.large_weight_initializer()
net.SGD(training_data, 10, 10, 3.0, evaluation_data=test_data, monitor_evaluation_accuracy=True)

Epoch 0 training complete
Accuracy on evaluation data: 8340 / 10000
Epoch 1 training complete
Accuracy on evaluation data: 8495 / 10000
Epoch 2 training complete
Accuracy on evaluation data: 9327 / 10000
Epoch 3 training complete
Accuracy on evaluation data: 9484 / 10000
Epoch 4 training complete
Accuracy on evaluation data: 9477 / 10000
Epoch 5 training complete
Accuracy on evaluation data: 9499 / 10000
Epoch 6 training complete
Accuracy on evaluation data: 9551 / 10000
Epoch 7 training complete
Accuracy on evaluation data: 9565 / 10000
Epoch 8 training complete
Accuracy on evaluation data: 9562 / 10000
Epoch 9 training complete
Accuracy on evaluation data: 9575 / 10000


([], [8340, 8495, 9327, 9484, 9477, 9499, 9551, 9565, 9562, 9575], [], [])

In [17]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net = Network([784, 100, 10], cost=CrossEntropyCost)
net.large_weight_initializer()
net.SGD(training_data, 10, 10, 0.5, evaluation_data=test_data, monitor_evaluation_accuracy=True)

Epoch 0 training complete
Accuracy on evaluation data: 9327 / 10000
Epoch 1 training complete
Accuracy on evaluation data: 9485 / 10000
Epoch 2 training complete
Accuracy on evaluation data: 9533 / 10000
Epoch 3 training complete
Accuracy on evaluation data: 9600 / 10000
Epoch 4 training complete
Accuracy on evaluation data: 9620 / 10000
Epoch 5 training complete
Accuracy on evaluation data: 9632 / 10000
Epoch 6 training complete
Accuracy on evaluation data: 9608 / 10000
Epoch 7 training complete
Accuracy on evaluation data: 9651 / 10000
Epoch 8 training complete
Accuracy on evaluation data: 9666 / 10000
Epoch 9 training complete
Accuracy on evaluation data: 9652 / 10000


([], [9327, 9485, 9533, 9600, 9620, 9632, 9608, 9651, 9666, 9652], [], [])

In [20]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net = Network([784, 100, 10], cost=CrossEntropyCost)
net.large_weight_initializer()
net.SGD(list(training_data)[:1000], 400, 10, 0.5,
        evaluation_data=list(test_data)[:1000],
        monitor_evaluation_accuracy=True,
        monitor_training_cost=True)

Epoch 0 training complete
Cost on training data: 1.4860090005504503
Accuracy on evaluation data: 617 / 1000
Epoch 1 training complete
Cost on training data: 0.8280943195441068
Accuracy on evaluation data: 694 / 1000
Epoch 2 training complete
Cost on training data: 0.5366466416989764
Accuracy on evaluation data: 767 / 1000
Epoch 3 training complete
Cost on training data: 0.403756159940483
Accuracy on evaluation data: 755 / 1000
Epoch 4 training complete
Cost on training data: 0.3085568322759454
Accuracy on evaluation data: 757 / 1000
Epoch 5 training complete
Cost on training data: 0.24047572413134766
Accuracy on evaluation data: 778 / 1000
Epoch 6 training complete
Cost on training data: 0.1734732106297779
Accuracy on evaluation data: 789 / 1000
Epoch 7 training complete
Cost on training data: 0.15306847764062245
Accuracy on evaluation data: 800 / 1000
Epoch 8 training complete
Cost on training data: 0.13873407517143088
Accuracy on evaluation data: 781 / 1000
Epoch 9 training complete


Accuracy on evaluation data: 813 / 1000
Epoch 75 training complete
Cost on training data: 0.00860857454495136
Accuracy on evaluation data: 812 / 1000
Epoch 76 training complete
Cost on training data: 0.00848546245568806
Accuracy on evaluation data: 814 / 1000
Epoch 77 training complete
Cost on training data: 0.008372491425575869
Accuracy on evaluation data: 813 / 1000
Epoch 78 training complete
Cost on training data: 0.008256239836670941
Accuracy on evaluation data: 816 / 1000
Epoch 79 training complete
Cost on training data: 0.008135202094298369
Accuracy on evaluation data: 812 / 1000
Epoch 80 training complete
Cost on training data: 0.008026750553486157
Accuracy on evaluation data: 815 / 1000
Epoch 81 training complete
Cost on training data: 0.007912340064243024
Accuracy on evaluation data: 813 / 1000
Epoch 82 training complete
Cost on training data: 0.007812776498783147
Accuracy on evaluation data: 812 / 1000
Epoch 83 training complete
Cost on training data: 0.007706704179971372
Acc

Accuracy on evaluation data: 816 / 1000
Epoch 149 training complete
Cost on training data: 0.004055702011084691
Accuracy on evaluation data: 815 / 1000
Epoch 150 training complete
Cost on training data: 0.004026487261793825
Accuracy on evaluation data: 815 / 1000
Epoch 151 training complete
Cost on training data: 0.003996695039947755
Accuracy on evaluation data: 816 / 1000
Epoch 152 training complete
Cost on training data: 0.003968026085300022
Accuracy on evaluation data: 815 / 1000
Epoch 153 training complete
Cost on training data: 0.003940351293773149
Accuracy on evaluation data: 817 / 1000
Epoch 154 training complete
Cost on training data: 0.003911257112385546
Accuracy on evaluation data: 815 / 1000
Epoch 155 training complete
Cost on training data: 0.003884147860876704
Accuracy on evaluation data: 814 / 1000
Epoch 156 training complete
Cost on training data: 0.003856710417840155
Accuracy on evaluation data: 816 / 1000
Epoch 157 training complete
Cost on training data: 0.00382995413

Accuracy on evaluation data: 818 / 1000
Epoch 222 training complete
Cost on training data: 0.0026169942435038795
Accuracy on evaluation data: 819 / 1000
Epoch 223 training complete
Cost on training data: 0.002604044933142382
Accuracy on evaluation data: 818 / 1000
Epoch 224 training complete
Cost on training data: 0.0025913000085283568
Accuracy on evaluation data: 818 / 1000
Epoch 225 training complete
Cost on training data: 0.0025786345594925658
Accuracy on evaluation data: 818 / 1000
Epoch 226 training complete
Cost on training data: 0.0025661289548268985
Accuracy on evaluation data: 819 / 1000
Epoch 227 training complete
Cost on training data: 0.0025536455671429903
Accuracy on evaluation data: 819 / 1000
Epoch 228 training complete
Cost on training data: 0.002541405357160823
Accuracy on evaluation data: 819 / 1000
Epoch 229 training complete
Cost on training data: 0.0025292313886642833
Accuracy on evaluation data: 820 / 1000
Epoch 230 training complete
Cost on training data: 0.00251

Accuracy on evaluation data: 821 / 1000
Epoch 295 training complete
Cost on training data: 0.0019163996527454892
Accuracy on evaluation data: 822 / 1000
Epoch 296 training complete
Cost on training data: 0.00190932912335846
Accuracy on evaluation data: 822 / 1000
Epoch 297 training complete
Cost on training data: 0.0019023088684398197
Accuracy on evaluation data: 822 / 1000
Epoch 298 training complete
Cost on training data: 0.001895342286152582
Accuracy on evaluation data: 822 / 1000
Epoch 299 training complete
Cost on training data: 0.0018884094379651888
Accuracy on evaluation data: 822 / 1000
Epoch 300 training complete
Cost on training data: 0.001881537622707725
Accuracy on evaluation data: 822 / 1000
Epoch 301 training complete
Cost on training data: 0.0018747695136692548
Accuracy on evaluation data: 822 / 1000
Epoch 302 training complete
Cost on training data: 0.0018679846481921593
Accuracy on evaluation data: 822 / 1000
Epoch 303 training complete
Cost on training data: 0.0018612

Epoch 368 training complete
Cost on training data: 0.0015060487230755943
Accuracy on evaluation data: 822 / 1000
Epoch 369 training complete
Cost on training data: 0.0015016142248072547
Accuracy on evaluation data: 822 / 1000
Epoch 370 training complete
Cost on training data: 0.0014971929437874525
Accuracy on evaluation data: 822 / 1000
Epoch 371 training complete
Cost on training data: 0.0014928024819473014
Accuracy on evaluation data: 822 / 1000
Epoch 372 training complete
Cost on training data: 0.0014884519822878641
Accuracy on evaluation data: 822 / 1000
Epoch 373 training complete
Cost on training data: 0.0014841199857266619
Accuracy on evaluation data: 822 / 1000
Epoch 374 training complete
Cost on training data: 0.0014797942902950877
Accuracy on evaluation data: 822 / 1000
Epoch 375 training complete
Cost on training data: 0.001475500330871552
Accuracy on evaluation data: 822 / 1000
Epoch 376 training complete
Cost on training data: 0.001471237886573365
Accuracy on evaluation da

([],
 [617,
  694,
  767,
  755,
  757,
  778,
  789,
  800,
  781,
  796,
  799,
  795,
  799,
  798,
  798,
  804,
  801,
  806,
  802,
  808,
  808,
  807,
  811,
  804,
  805,
  805,
  810,
  805,
  808,
  809,
  808,
  812,
  810,
  810,
  811,
  810,
  809,
  811,
  814,
  810,
  809,
  810,
  811,
  811,
  811,
  811,
  811,
  811,
  809,
  812,
  813,
  814,
  813,
  813,
  813,
  811,
  816,
  814,
  814,
  813,
  815,
  812,
  813,
  815,
  813,
  815,
  815,
  812,
  815,
  813,
  813,
  814,
  815,
  813,
  813,
  812,
  814,
  813,
  816,
  812,
  815,
  813,
  812,
  812,
  813,
  813,
  814,
  813,
  813,
  812,
  812,
  814,
  813,
  813,
  813,
  813,
  812,
  812,
  813,
  812,
  813,
  814,
  815,
  812,
  812,
  813,
  815,
  814,
  814,
  815,
  813,
  815,
  813,
  813,
  814,
  814,
  813,
  815,
  814,
  813,
  815,
  813,
  814,
  815,
  814,
  814,
  813,
  814,
  813,
  813,
  814,
  814,
  813,
  813,
  814,
  814,
  814,
  814,
  815,
  814,
  815,
  814,
 